In [3]:
import requests
import pandas as pd
import requests
from lxml import etree

In [4]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

In [5]:
#Get each university link, names,locations and general infos(public or private? Established Years) (127 in total)
Uni_links=[]
Uni_names=[]
Uni_locas=[]
Uni_general_infos=[]
for i in range(1,46):
    #Home Page link
    url='https://studyabroad.shiksha.com/usa/universities-{}'.format(i)
    res=requests.get(url,headers=headers)
    html=etree.HTML(res.text)
    #Get Each U's Link
    Uni_link=html.xpath('//tr/td[2]/p/a/@href')
    Uni_name=html.xpath('//tr/td[2]/p/a/strong/text()')
    Uni_loca=html.xpath('//tr[@class]/td[3]/text()')
    Uni_general_info=html.xpath('//tr[@class]/td[4]/p[1]/text()')
    Uni_links.extend(Uni_link)
    Uni_names.extend(Uni_name)
    Uni_locas.extend(Uni_loca)
    Uni_general_infos.extend(Uni_general_info)

In [6]:
#Construct URLs of course page of each university
Uni_courses=[]
for i in Uni_links:
    Uni_course=i+'/courses'
    Uni_courses.append(Uni_course)

In [10]:
#Iterate each course page,get all program names and links of each university
Unis=[]
Locations=[]
Uni_informations=[]
Programs=[]
Program_links=[]
for i,j,h,m in zip(Uni_courses,Uni_names,Uni_locas,Uni_general_infos):
    res1=requests.get(i,headers=headers)
    html1=etree.HTML(res1.text)
    Uni_program=html1.xpath('//div[@class="Styled__AllCourseTuple-sc-1yl1nt-33 kHvhnS"]/a/@title')
    partial_link=html1.xpath('//div[@class="Styled__AllCourseTuple-sc-1yl1nt-33 kHvhnS"]/a/@href')# link not complete
    Program_link=['https://studyabroad.shiksha.com'+link for link in partial_link ]
    
    for n in Uni_program:
        Unis.append(j)
        Locations.append(h)
        Uni_informations.append(m)
    Programs.extend(Uni_program)
    Program_links.extend(Program_link)

In [11]:
print(len(Programs))
print(len(Program_links))

8766
8766


In [12]:
#Iterate each program link to get Basic Information(Program Length, Level, GPA)
Durations=[]
Levels=[]
Grades=[]
#Language and Standard Tests Requirements 
Toefls=[]
Toefls_adds=[]
Ielts=[]
Ielts_adds=[]
Ptes=[]
Ptes_adds=[]
Gres=[]
Gres_adds=[]
Gmats=[]
Gmats_adds=[]
#Course Description, Academic Requirements, Work Experience Requirements and Additional Requirements
Course_des=[]
Ad_res=[]
Work_res=[]
Work_adds=[]
Addition_res=[]
for i,j in zip(Program_links,range(len(Program_links))):
    res2=requests.get(i,headers=headers)
    html2=etree.HTML(res2.text)
    
    #Program Duration
    try: 
        Duration=html2.xpath('//table[@class="Styled__TableStyle-sc-10ucg51-0 lfnXDf"]/tbody/tr[1]/td[2]/text()')
        Durations.append(Duration[0])
    except:
        Durations.append('Not Avaliable')
        
    #Program Level    
    try:
        Level=html2.xpath('//table[@class="Styled__TableStyle-sc-10ucg51-0 lfnXDf"]/tbody/tr[2]/td[2]/text()')
        Levels.append(Level[0])
    except:
        Levels.append('Not Avaliable')
        
    #Entry Grades Requirements  
    try:
        Grade=html2.xpath('//div[@class="Styled__EntryReqDetails-sc-14wej5r-4 eGPqdX"]/span/text()')
        Grades.append(Grade[0])
    except:
        Grades.append('Not Avaliable')
        
    #TOEFL
    try:
        Toefl=html2.xpath("//label[text()='TOEFL :']/following-sibling::*/text()")
        if len(Toefl[0])>20:
            Toefls.append('Error Type')
        else:
            Toefls.append(Toefl[0])
    except:
        Toefls.append('Not Available')
    #TOEFL Additional Requirements
    try:
        t=html2.xpath("//label[text()='TOEFL :']/../div/ul[1]/li/text()")
        Toefls_adds.append(t[0].replace('\xa0',''))
    except:
        Toefls_adds.append('Not Available')
        
    #IELTS
    try:
        Ielt=html2.xpath("//label[text()='IELTS :']/following-sibling::*/text()")
        if len(Ielt[0])>20:
            Ielts.append('Error Type')
        else:
            Ielts.append(Ielt[0])
    except:
        Ielts.append('Not Available')
    #IELTS Additional Requirements
    try:
        ii=html2.xpath("//label[text()='IELTS :']/../div/ul[1]/li/text()")
        Ielts_adds.append(ii[0].replace('\xa0',''))
    except:
        Ielts_adds.append('Not Available')
    
    #PTE
    try:
        Pte=html2.xpath("//label[text()='PTE :']/following-sibling::*/text()")
        if len(Pte[0])>20:
            Ptes.append('Error Type')
        else:
            Ptes.append(Pte[0])
    except:
        Ptes.append('Not Available')
    #PTE Additional Requirements
    try:
        p=html2.xpath("//label[text()='PTE :']/../div/ul[1]/li/text()")
        Ptes_adds.append(p[0].replace('\xa0',''))
    except:
        Ptes_adds.append('Not Available')
        
    #GRE
    try:
        Gre=html2.xpath("//label[text()='GRE :']/following-sibling::*/text()")
        if len(Gre[0])>20:
            Gres.append('Error Type')
        else:
            Gres.append(Gre[0])
    except:
        Gres.append('Not Available')
    #GRE Additional Requirements
    try:
        gre=html2.xpath("//label[text()='GRE :']/../div/ul[1]/li/text()")
        Gres_adds.append(gre[0].replace('\xa0',''))
    except:
        Gres_adds.append('Not Available')

    #GMAT
    try:
        Gmat=html2.xpath("//label[text()='GMAT :']/following-sibling::*/text()")
        if len(Gmat[0])>20:
            Gmats.append('Error Type')
        else:
            Gmats.append(Gmat[0])
    except:
        Gmats.append('Not Available')
    #GMAT Additional Requirements
    try:
        gmat=html2.xpath("//label[text()='GMAT :']/../div/ul[1]/li/text()")
        Gmats_adds.append(gmat[0].replace('\xa0',''))
    except:
        Gmats_adds.append('Not Available')
        
    #Course Description
    try:
        Course_de=["".join(html2.xpath('//div[@class="Styled__CourseDescription-sc-14wej5r-0 Gtoss"]/div[1]/ul//li/text()'))][0].replace('\n',' ').replace('\xa0',' ')
        Course_des.append(Course_de)
    except:
        Course_des.append('Not Available')
        
    #Entry Academic Requirements
    try:
        Ad_re=["".join(html2.xpath('//div[@class="Styled__EntryReqDetails-sc-14wej5r-4 eGPqdX"]/div[@class="default_CommentTxt"]/ul//text()'))][0].replace('\n',' ').replace('\xa0',' ')
        Ad_res.append(Ad_re)
    except:
        Ad_res.append('Not Avaliable')
    
    #Work Experience Requirements
    try:
        Work_re=html2.xpath("//td[text()='Work experience']/../td[2]/div/span/text()")
        Work_res.append(Work_re[0])
    except:
        Work_res.append('Not Available')
    
    #Work Experience Detailed Requirements
    try:
        Work_add=["".join(html2.xpath("//td[text()='Work experience']/../td[2]/div/div[@class='default_CommentTxt']/ul//text()"))][0].replace('\n',' ').replace('\xa0',' ')
        Work_adds.append(Work_add)
    except:
        Work_adds.append('Not Available')
    
    #Additional Requirements
    try:
        Addition_re=["".join(html2.xpath("//td[text()='Additional info']/../td[2]/div/div[@class='default_CommentTxt']/ul//text()"))][0].replace('\n',' ').replace('\xa0',' ')
        Addition_res.append(Addition_re)
    except:
        Addition_res.append('Not Available')

    print('{} Programs Finished'.format(j+1))

1 Programs Finished
2 Programs Finished
3 Programs Finished
4 Programs Finished
5 Programs Finished
6 Programs Finished
7 Programs Finished
8 Programs Finished
9 Programs Finished
10 Programs Finished
11 Programs Finished
12 Programs Finished
13 Programs Finished
14 Programs Finished
15 Programs Finished
16 Programs Finished
17 Programs Finished
18 Programs Finished
19 Programs Finished
20 Programs Finished
21 Programs Finished
22 Programs Finished
23 Programs Finished
24 Programs Finished
25 Programs Finished
26 Programs Finished
27 Programs Finished
28 Programs Finished
29 Programs Finished
30 Programs Finished
31 Programs Finished
32 Programs Finished
33 Programs Finished
34 Programs Finished
35 Programs Finished
36 Programs Finished
37 Programs Finished
38 Programs Finished
39 Programs Finished
40 Programs Finished
41 Programs Finished
42 Programs Finished
43 Programs Finished
44 Programs Finished
45 Programs Finished
46 Programs Finished
47 Programs Finished
48 Programs Finished
4

379 Programs Finished
380 Programs Finished
381 Programs Finished
382 Programs Finished
383 Programs Finished
384 Programs Finished
385 Programs Finished
386 Programs Finished
387 Programs Finished
388 Programs Finished
389 Programs Finished
390 Programs Finished
391 Programs Finished
392 Programs Finished
393 Programs Finished
394 Programs Finished
395 Programs Finished
396 Programs Finished
397 Programs Finished
398 Programs Finished
399 Programs Finished
400 Programs Finished
401 Programs Finished
402 Programs Finished
403 Programs Finished
404 Programs Finished
405 Programs Finished
406 Programs Finished
407 Programs Finished
408 Programs Finished
409 Programs Finished
410 Programs Finished
411 Programs Finished
412 Programs Finished
413 Programs Finished
414 Programs Finished
415 Programs Finished
416 Programs Finished
417 Programs Finished
418 Programs Finished
419 Programs Finished
420 Programs Finished
421 Programs Finished
422 Programs Finished
423 Programs Finished
424 Progra

752 Programs Finished
753 Programs Finished
754 Programs Finished
755 Programs Finished
756 Programs Finished
757 Programs Finished
758 Programs Finished
759 Programs Finished
760 Programs Finished
761 Programs Finished
762 Programs Finished
763 Programs Finished
764 Programs Finished
765 Programs Finished
766 Programs Finished
767 Programs Finished
768 Programs Finished
769 Programs Finished
770 Programs Finished
771 Programs Finished
772 Programs Finished
773 Programs Finished
774 Programs Finished
775 Programs Finished
776 Programs Finished
777 Programs Finished
778 Programs Finished
779 Programs Finished
780 Programs Finished
781 Programs Finished
782 Programs Finished
783 Programs Finished
784 Programs Finished
785 Programs Finished
786 Programs Finished
787 Programs Finished
788 Programs Finished
789 Programs Finished
790 Programs Finished
791 Programs Finished
792 Programs Finished
793 Programs Finished
794 Programs Finished
795 Programs Finished
796 Programs Finished
797 Progra

1119 Programs Finished
1120 Programs Finished
1121 Programs Finished
1122 Programs Finished
1123 Programs Finished
1124 Programs Finished
1125 Programs Finished
1126 Programs Finished
1127 Programs Finished
1128 Programs Finished
1129 Programs Finished
1130 Programs Finished
1131 Programs Finished
1132 Programs Finished
1133 Programs Finished
1134 Programs Finished
1135 Programs Finished
1136 Programs Finished
1137 Programs Finished
1138 Programs Finished
1139 Programs Finished
1140 Programs Finished
1141 Programs Finished
1142 Programs Finished
1143 Programs Finished
1144 Programs Finished
1145 Programs Finished
1146 Programs Finished
1147 Programs Finished
1148 Programs Finished
1149 Programs Finished
1150 Programs Finished
1151 Programs Finished
1152 Programs Finished
1153 Programs Finished
1154 Programs Finished
1155 Programs Finished
1156 Programs Finished
1157 Programs Finished
1158 Programs Finished
1159 Programs Finished
1160 Programs Finished
1161 Programs Finished
1162 Progra

1476 Programs Finished
1477 Programs Finished
1478 Programs Finished
1479 Programs Finished
1480 Programs Finished
1481 Programs Finished
1482 Programs Finished
1483 Programs Finished
1484 Programs Finished
1485 Programs Finished
1486 Programs Finished
1487 Programs Finished
1488 Programs Finished
1489 Programs Finished
1490 Programs Finished
1491 Programs Finished
1492 Programs Finished
1493 Programs Finished
1494 Programs Finished
1495 Programs Finished
1496 Programs Finished
1497 Programs Finished
1498 Programs Finished
1499 Programs Finished
1500 Programs Finished
1501 Programs Finished
1502 Programs Finished
1503 Programs Finished
1504 Programs Finished
1505 Programs Finished
1506 Programs Finished
1507 Programs Finished
1508 Programs Finished
1509 Programs Finished
1510 Programs Finished
1511 Programs Finished
1512 Programs Finished
1513 Programs Finished
1514 Programs Finished
1515 Programs Finished
1516 Programs Finished
1517 Programs Finished
1518 Programs Finished
1519 Progra

...

6117 Programs Finished
6118 Programs Finished
6119 Programs Finished
6120 Programs Finished
6121 Programs Finished
6122 Programs Finished
6123 Programs Finished
6124 Programs Finished
6125 Programs Finished
6126 Programs Finished
6127 Programs Finished
6128 Programs Finished
6129 Programs Finished
6130 Programs Finished
6131 Programs Finished
6132 Programs Finished
6133 Programs Finished
6134 Programs Finished
6135 Programs Finished
6136 Programs Finished
6137 Programs Finished
6138 Programs Finished
6139 Programs Finished
6140 Programs Finished
6141 Programs Finished
6142 Programs Finished
6143 Programs Finished
6144 Programs Finished
6145 Programs Finished
6146 Programs Finished
6147 Programs Finished
6148 Programs Finished
6149 Programs Finished
6150 Programs Finished
6151 Programs Finished
6152 Programs Finished
6153 Programs Finished
6154 Programs Finished
6155 Programs Finished
6156 Programs Finished
6157 Programs Finished
6158 Programs Finished
6159 Programs Finished
6160 Progra

6474 Programs Finished
6475 Programs Finished
6476 Programs Finished
6477 Programs Finished
6478 Programs Finished
6479 Programs Finished
6480 Programs Finished
6481 Programs Finished
6482 Programs Finished
6483 Programs Finished
6484 Programs Finished
6485 Programs Finished
6486 Programs Finished
6487 Programs Finished
6488 Programs Finished
6489 Programs Finished
6490 Programs Finished
6491 Programs Finished
6492 Programs Finished
6493 Programs Finished
6494 Programs Finished
6495 Programs Finished
6496 Programs Finished
6497 Programs Finished
6498 Programs Finished
6499 Programs Finished
6500 Programs Finished
6501 Programs Finished
6502 Programs Finished
6503 Programs Finished
6504 Programs Finished
6505 Programs Finished
6506 Programs Finished
6507 Programs Finished
6508 Programs Finished
6509 Programs Finished
6510 Programs Finished
6511 Programs Finished
6512 Programs Finished
6513 Programs Finished
6514 Programs Finished
6515 Programs Finished
6516 Programs Finished
6517 Progra

6831 Programs Finished
6832 Programs Finished
6833 Programs Finished
6834 Programs Finished
6835 Programs Finished
6836 Programs Finished
6837 Programs Finished
6838 Programs Finished
6839 Programs Finished
6840 Programs Finished
6841 Programs Finished
6842 Programs Finished
6843 Programs Finished
6844 Programs Finished
6845 Programs Finished
6846 Programs Finished
6847 Programs Finished
6848 Programs Finished
6849 Programs Finished
6850 Programs Finished
6851 Programs Finished
6852 Programs Finished
6853 Programs Finished
6854 Programs Finished
6855 Programs Finished
6856 Programs Finished
6857 Programs Finished
6858 Programs Finished
6859 Programs Finished
6860 Programs Finished
6861 Programs Finished
6862 Programs Finished
6863 Programs Finished
6864 Programs Finished
6865 Programs Finished
6866 Programs Finished
6867 Programs Finished
6868 Programs Finished
6869 Programs Finished
6870 Programs Finished
6871 Programs Finished
6872 Programs Finished
6873 Programs Finished
6874 Progra

7188 Programs Finished
7189 Programs Finished
7190 Programs Finished
7191 Programs Finished
7192 Programs Finished
7193 Programs Finished
7194 Programs Finished
7195 Programs Finished
7196 Programs Finished
7197 Programs Finished
7198 Programs Finished
7199 Programs Finished
7200 Programs Finished
7201 Programs Finished
7202 Programs Finished
7203 Programs Finished
7204 Programs Finished
7205 Programs Finished
7206 Programs Finished
7207 Programs Finished
7208 Programs Finished
7209 Programs Finished
7210 Programs Finished
7211 Programs Finished
7212 Programs Finished
7213 Programs Finished
7214 Programs Finished
7215 Programs Finished
7216 Programs Finished
7217 Programs Finished
7218 Programs Finished
7219 Programs Finished
7220 Programs Finished
7221 Programs Finished
7222 Programs Finished
7223 Programs Finished
7224 Programs Finished
7225 Programs Finished
7226 Programs Finished
7227 Programs Finished
7228 Programs Finished
7229 Programs Finished
7230 Programs Finished
7231 Progra

ProxyError: HTTPSConnectionPool(host='studyabroad.shiksha.com', port=443): Max retries exceeded with url: /usa/universities/miami-dade-college/supply-chain-management-bachelor-of-applied-science (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))

In [29]:
#continued 
for i,j in zip(Program_links[7334:],range(len(Program_links[7334:]))):
    res2=requests.get(i,headers=headers)
    html2=etree.HTML(res2.text)
    
    #Program Duration
    try: 
        Duration=html2.xpath('//table[@class="Styled__TableStyle-sc-10ucg51-0 lfnXDf"]/tbody/tr[1]/td[2]/text()')
        Durations.append(Duration[0])
    except:
        Durations.append('Not Avaliable')
        
    #Program Level    
    try:
        Level=html2.xpath('//table[@class="Styled__TableStyle-sc-10ucg51-0 lfnXDf"]/tbody/tr[2]/td[2]/text()')
        Levels.append(Level[0])
    except:
        Levels.append('Not Avaliable')
        
    #Entry Grades Requirements  
    try:
        Grade=html2.xpath('//div[@class="Styled__EntryReqDetails-sc-14wej5r-4 eGPqdX"]/span/text()')
        Grades.append(Grade[0])
    except:
        Grades.append('Not Avaliable')
        
    #TOEFL
    try:
        Toefl=html2.xpath("//label[text()='TOEFL :']/following-sibling::*/text()")
        if len(Toefl[0])>20:
            Toefls.append('Error Type')
        else:
            Toefls.append(Toefl[0])
    except:
        Toefls.append('Not Available')
    #TOEFL Additional Requirements
    try:
        t=html2.xpath("//label[text()='TOEFL :']/../div/ul[1]/li/text()")
        Toefls_adds.append(t[0].replace('\xa0',''))
    except:
        Toefls_adds.append('Not Available')
        
    #IELTS
    try:
        Ielt=html2.xpath("//label[text()='IELTS :']/following-sibling::*/text()")
        if len(Ielt[0])>20:
            Ielts.append('Error Type')
        else:
            Ielts.append(Ielt[0])
    except:
        Ielts.append('Not Available')
    #IELTS Additional Requirements
    try:
        ii=html2.xpath("//label[text()='IELTS :']/../div/ul[1]/li/text()")
        Ielts_adds.append(ii[0].replace('\xa0',''))
    except:
        Ielts_adds.append('Not Available')
    
    #PTE
    try:
        Pte=html2.xpath("//label[text()='PTE :']/following-sibling::*/text()")
        if len(Pte[0])>20:
            Ptes.append('Error Type')
        else:
            Ptes.append(Pte[0])
    except:
        Ptes.append('Not Available')
    #PTE Additional Requirements
    try:
        p=html2.xpath("//label[text()='PTE :']/../div/ul[1]/li/text()")
        Ptes_adds.append(p[0].replace('\xa0',''))
    except:
        Ptes_adds.append('Not Available')
        
    #GRE
    try:
        Gre=html2.xpath("//label[text()='GRE :']/following-sibling::*/text()")
        if len(Gre[0])>20:
            Gres.append('Error Type')
        else:
            Gres.append(Gre[0])
    except:
        Gres.append('Not Available')
    #GRE Additional Requirements
    try:
        gre=html2.xpath("//label[text()='GRE :']/../div/ul[1]/li/text()")
        Gres_adds.append(gre[0].replace('\xa0',''))
    except:
        Gres_adds.append('Not Available')

    #GMAT
    try:
        Gmat=html2.xpath("//label[text()='GMAT :']/following-sibling::*/text()")
        if len(Gmat[0])>20:
            Gmats.append('Error Type')
        else:
            Gmats.append(Gmat[0])
    except:
        Gmats.append('Not Available')
    #GMAT Additional Requirements
    try:
        gmat=html2.xpath("//label[text()='GMAT :']/../div/ul[1]/li/text()")
        Gmats_adds.append(gmat[0].replace('\xa0',''))
    except:
        Gmats_adds.append('Not Available')
        
    #Course Description
    try:
        Course_de=["".join(html2.xpath('//div[@class="Styled__CourseDescription-sc-14wej5r-0 Gtoss"]/div[1]/ul//li/text()'))][0].replace('\n',' ').replace('\xa0',' ')
        Course_des.append(Course_de)
    except:
        Course_des.append('Not Available')
        
    #Entry Academic Requirements
    try:
        Ad_re=["".join(html2.xpath('//div[@class="Styled__EntryReqDetails-sc-14wej5r-4 eGPqdX"]/div[@class="default_CommentTxt"]/ul//text()'))][0].replace('\n',' ').replace('\xa0',' ')
        Ad_res.append(Ad_re)
    except:
        Ad_res.append('Not Avaliable')
    
    #Work Experience Requirements
    try:
        Work_re=html2.xpath("//td[text()='Work experience']/../td[2]/div/span/text()")
        Work_res.append(Work_re[0])
    except:
        Work_res.append('Not Available')
    
    #Work Experience Detailed Requirements
    try:
        Work_add=["".join(html2.xpath("//td[text()='Work experience']/../td[2]/div/div[@class='default_CommentTxt']/ul//text()"))][0].replace('\n',' ').replace('\xa0',' ')
        Work_adds.append(Work_add)
    except:
        Work_adds.append('Not Available')
    
    #Additional Requirements
    try:
        Addition_re=["".join(html2.xpath("//td[text()='Additional info']/../td[2]/div/div[@class='default_CommentTxt']/ul//text()"))][0].replace('\n',' ').replace('\xa0',' ')
        Addition_res.append(Addition_re)
    except:
        Addition_res.append('Not Available')

    print('{} Programs Finished'.format(j+1))

1 Programs Finished
2 Programs Finished
3 Programs Finished
4 Programs Finished
5 Programs Finished
6 Programs Finished
7 Programs Finished
8 Programs Finished
9 Programs Finished
10 Programs Finished
11 Programs Finished
12 Programs Finished
13 Programs Finished
14 Programs Finished
15 Programs Finished
16 Programs Finished
17 Programs Finished
18 Programs Finished
19 Programs Finished
20 Programs Finished
21 Programs Finished
22 Programs Finished
23 Programs Finished
24 Programs Finished
25 Programs Finished
26 Programs Finished
27 Programs Finished
28 Programs Finished
29 Programs Finished
30 Programs Finished
31 Programs Finished
32 Programs Finished
33 Programs Finished
34 Programs Finished
35 Programs Finished
36 Programs Finished
37 Programs Finished
38 Programs Finished
39 Programs Finished
40 Programs Finished
41 Programs Finished
42 Programs Finished
43 Programs Finished
44 Programs Finished
45 Programs Finished
46 Programs Finished
47 Programs Finished
48 Programs Finished
4

379 Programs Finished
380 Programs Finished
381 Programs Finished
382 Programs Finished
383 Programs Finished
384 Programs Finished
385 Programs Finished
386 Programs Finished
387 Programs Finished
388 Programs Finished
389 Programs Finished
390 Programs Finished
391 Programs Finished
392 Programs Finished
393 Programs Finished
394 Programs Finished
395 Programs Finished
396 Programs Finished
397 Programs Finished
398 Programs Finished
399 Programs Finished
400 Programs Finished
401 Programs Finished
402 Programs Finished
403 Programs Finished
404 Programs Finished
405 Programs Finished
406 Programs Finished
407 Programs Finished
408 Programs Finished
409 Programs Finished
410 Programs Finished
411 Programs Finished
412 Programs Finished
413 Programs Finished
414 Programs Finished
415 Programs Finished
416 Programs Finished
417 Programs Finished
418 Programs Finished
419 Programs Finished
420 Programs Finished
421 Programs Finished
422 Programs Finished
423 Programs Finished
424 Progra

752 Programs Finished
753 Programs Finished
754 Programs Finished
755 Programs Finished
756 Programs Finished
757 Programs Finished
758 Programs Finished
759 Programs Finished
760 Programs Finished
761 Programs Finished
762 Programs Finished
763 Programs Finished
764 Programs Finished
765 Programs Finished
766 Programs Finished
767 Programs Finished
768 Programs Finished
769 Programs Finished
770 Programs Finished
771 Programs Finished
772 Programs Finished
773 Programs Finished
774 Programs Finished
775 Programs Finished
776 Programs Finished
777 Programs Finished
778 Programs Finished
779 Programs Finished
780 Programs Finished
781 Programs Finished
782 Programs Finished
783 Programs Finished
784 Programs Finished
785 Programs Finished
786 Programs Finished
787 Programs Finished
788 Programs Finished
789 Programs Finished
790 Programs Finished
791 Programs Finished
792 Programs Finished
793 Programs Finished
794 Programs Finished
795 Programs Finished
796 Programs Finished
797 Progra

1119 Programs Finished
1120 Programs Finished
1121 Programs Finished
1122 Programs Finished
1123 Programs Finished
1124 Programs Finished
1125 Programs Finished
1126 Programs Finished
1127 Programs Finished
1128 Programs Finished
1129 Programs Finished
1130 Programs Finished
1131 Programs Finished
1132 Programs Finished
1133 Programs Finished
1134 Programs Finished
1135 Programs Finished
1136 Programs Finished
1137 Programs Finished
1138 Programs Finished
1139 Programs Finished
1140 Programs Finished
1141 Programs Finished
1142 Programs Finished
1143 Programs Finished
1144 Programs Finished
1145 Programs Finished
1146 Programs Finished
1147 Programs Finished
1148 Programs Finished
1149 Programs Finished
1150 Programs Finished
1151 Programs Finished
1152 Programs Finished
1153 Programs Finished
1154 Programs Finished
1155 Programs Finished
1156 Programs Finished
1157 Programs Finished
1158 Programs Finished
1159 Programs Finished
1160 Programs Finished
1161 Programs Finished
1162 Progra

In [33]:
Program_info=pd.DataFrame()
Program_info['University']=Unis
Program_info['Location']=Locations
Program_info['Property and Estd Year']=Uni_informations
Program_info['Programs']=Programs
Program_info['Programs Description']=Course_des
Program_info['Durations']=Durations
Program_info['Levels']=Levels
Program_info['Entry GPA']=Grades
Program_info['Academic Requirements']=Ad_res
Program_info['Work Experience Requirements']=Work_res
Program_info['Work Experience Detailed Requirements']=Work_adds
Program_info['Additional Requirements']=Addition_res
Program_info['TOEFLS']=Toefls
Program_info['TOEFLS Details']=Toefls_adds
Program_info['IELTS']=Ielts
Program_info['IELTS Details']=Ielts_adds
Program_info['PTE']=Ptes
Program_info['PTE Details']=Ptes_adds
Program_info['GRE']=Gres
Program_info['GRE Details']=Gres_adds
Program_info['GMAT']=Gmats
Program_info['GMAT Details']=Gmats_adds
Program_info['Program_Links']=Program_links

In [34]:
Program_info.to_excel(r'C:\Users\wangchu\Desktop\Chu-USA University Programs Info.xlsx')